In [1]:
import os
import time
import random
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.optim as optim
from sklearn.model_selection import train_test_split

from Segmentation_models.UNet_3Plus import UNet_3Plus_DeepSup_CGM
from Segmentation_loss.iouLoss import IOU_loss
from Segmentation_loss.msssimLoss import msssim
from Segmentation_loss.bceLoss import BCE_loss

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data_dir, train, transform=None):
        self.data_dir = data_dir
        self.data_path = np.array(os.listdir(data_dir + '/images'))
        #self.label_path = np.array(os.listdir(data_dir + '/masks'))
        #self.data_dir = data_dir
        #self.data_path = data_path
        self.transform = transform
        if train:
            idx, _ = train_test_split(list(range(len(self.data_path))), test_size=0.2, random_state=1)
        else:
            _, idx = train_test_split(list(range(len(self.data_path))), test_size=0.2, random_state=1)
        self.data_path = self.data_path[idx]
    
    def __getitem__(self, index): 
        ### Begin your code ###
        file_name = self.data_path[index]
        path_image = self.data_dir + "/images/" + file_name
        path_label = self.data_dir + "/masks/" + file_name
        image = Image.open(path_image)
        label = Image.open(path_label)
        if self.transform is not None:
            image = self.transform(image)
            label = self.transform(label)
        
        # Data Augmentation
        augType = random.choice([0, 1, 2])
        if augType != 0:
            image = self.augment(image, augType)
            label = self.augment(label, augType)
        return image, label
        ### End your code ###

    def __len__(self): 
        '''return the size of the dataset'''
        ### Begin your code ###
        return(len(self.data_path))
        ### End your code ###
    
    def augment(self, image, augType):
        if augType == 1:
            aug_transform = transforms.Compose([transforms.RandomRotation(degrees=45)])
        if augType == 2:
            aug_transform = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5)])
        data = aug_transform(image)
        return data

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("\nRunning on:", device)

if device == 'cuda':
    device_name = torch.cuda.get_device_name()
    print("The device name is:", device_name)
    cap = torch.cuda.get_device_capability(device=None)
    print("The capability of this device is:", cap, '\n')

# hyper-parameters
seed = 1
MAX_EPOCH = 30
LR = 0.001
weight_decay = 1e-3
data_dir = 'segmented-images'

set_seed(seed)
print('random seed:', seed)


Running on: cpu
random seed: 1


In [3]:
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
train_data = MyDataset(data_dir=data_dir, train=True, transform=transform)
valid_data = MyDataset(data_dir=data_dir, train=False, transform=transform)
train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=1)

In [4]:
net = UNet_3Plus_DeepSup_CGM()
net.to(device)

optimizer = optim.Adam(net.parameters(), weight_decay=weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=MAX_EPOCH, eta_min=0, last_epoch=-1)

In [5]:
for image, label in train_loader:
    break

In [6]:
label = label[:,0,:,:].reshape([1] + list(label[:,0,:,:].shape))
label

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [11]:
label[label<0.5] = 0
label[label>=0.5] = 1
label

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [13]:
output = net(image)

D:\Anaconda3\lib\site-packages\torch\nn\functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
D:\Anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [18]:
loss_iou = IOU_loss(output, label)
loss_iou

tensor(0.9517, grad_fn=<DivBackward0>)

In [19]:
2 * loss_iou / (loss_iou + 1)

tensor(0.9752, grad_fn=<DivBackward0>)

In [23]:
output[output<0.5] = 0
output[output>=0.5] = 1
output

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]], grad_fn=<IndexPutBackward0>)

In [33]:
dice = 2 * torch.sum(output == label) / (256*256*2)

In [38]:
Iand1 = torch.sum(label[0,:,:,:]*output[0,:,:,:])
Ior1 = torch.sum(label[0,:,:,:]) + torch.sum(output[0,:,:,:])-Iand1
Iand1/Ior1

tensor(0.0508, grad_fn=<DivBackward0>)

In [39]:
IOU_loss(output, label)

tensor(0.9492, grad_fn=<DivBackward0>)

In [41]:
def soft_dice_loss(y_true, y_pred, epsilon=1e-6):
    nume = 2 * torch.sum(y_true * y_pred)
    deno = torch.sum(torch.square(y_pred) + torch.square(y_true))
    return 1-torch.mean(nume/(deno + epsilon))

In [42]:
soft_dice_loss(label, output)

tensor(0.9033, grad_fn=<RsubBackward1>)

In [45]:
dice = torch.sum(output == label) / (256*256)
dice

tensor(0.0508)

In [57]:
axes = tuple(range(1,len(output.shape)-1))
1 - torch.mean(2 * torch.sum(output*label, axes) / (torch.sum(torch.square(output) + torch.square(label), axes) + 1e-6))

tensor(0.9165, grad_fn=<RsubBackward1>)

In [58]:
soft_dice_loss(label, output)

tensor(0.9033, grad_fn=<RsubBackward1>)

In [60]:
criterion = nn.BCELoss()
criterion(output, label)

tensor(94.9219, grad_fn=<BinaryCrossEntropyBackward0>)

In [52]:
torch.sum(output*label)

tensor(3328., grad_fn=<SumBackward0>)

In [27]:
for epoch in range(1, MAX_EPOCH + 1):
    # Training
    loss_trn = 0.; dice_trn = 0.; iter_num = 0
    pred_trn = []; label_trn = []
    net.train()
    for image, label in train_loader:
        iter_num += 1

        image = image.to(device)
        label = label[:,0,:,:].reshape([1] + list(label[:,0,:,:].shape))
        label = label.to(device)
        pred = net(image)

        if epoch == MAX_EPOCH:
            pred_trn.append(pred); label_trn.append(label)

        optimizer.zero_grad()
        loss_iou = IOU_loss(pred, label)
        #print(loss_iou)
        #loss_mss = msssim(pred, label)
        #print(loss_mss)
        #loss_bce = BCE_loss(pred, label)
        #print(loss_bce)
        loss = loss_iou #+ loss_mss + loss_bce

        loss.backward()
        optimizer.step()

        loss_trn += loss.item()
        dice_trn += 2 * loss_iou / (loss_iou + 1)

    # print log
    print("Training: Epoch[{:0>3}/{:0>3}], Loss: {:.4f} Dice:{:.2%}".format(
            epoch, MAX_EPOCH, loss_trn / iter_num, dice_trn / iter_num))

tensor(0.9749, grad_fn=<DivBackward0>)
tensor(0.0426, grad_fn=<ProdBackward0>)
tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.9466, grad_fn=<DivBackward0>)
tensor(0.0202, grad_fn=<ProdBackward0>)
tensor(0.9362, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.9824, grad_fn=<DivBackward0>)
tensor(0.0153, grad_fn=<ProdBackward0>)
tensor(0.7058, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<ProdBackward0>)


RuntimeError: all elements of input should be between 0 and 1

In [ ]:
def main():
    transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
    train_data = MyDataset(data_dir=data_dir, train=True, transform=transform)
    valid_data = MyDataset(data_dir=data_dir, train=False, transform=transform)
    train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)
    valid_loader = DataLoader(dataset=valid_data, batch_size=1)

    net = UNet_3Plus_DeepSup_CGM()
    net.to(device)
    
    optimizer = optim.Adam(net.parameters(), weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=MAX_EPOCH, eta_min=0, last_epoch=-1)

    print('\nTraining start!\n')
    start = time.time()

    for epoch in range(1, MAX_EPOCH + 1):
        # Training
        loss_trn = 0.; dice_trn = 0.; iter_num = 0
        pred_trn = []; label_trn = []
        net.train()
        for image, label in train_loader:
            iter_num += 1
            
            image = image.to(device)
            label = label[:,0,:,:].reshape([1] + list(label[:,0,:,:].shape))
            label = label.to(device)
            output = net(image)
            pred = output[0]
            
            if epoch == MAX_EPOCH:
                    pred_trn.append(pred); label_trn.append(label)

            optimizer.zero_grad()
            loss_iou = IOU_loss(pred, label)
            loss = loss_iou #+ msssim(pred, label) + BCE_loss(pred, label)
            loss.backward()
            optimizer.step()
            
            loss_trn += loss.item()
            dice_trn += 2 * loss_iou / (loss_iou + 1)
        
        # print log
        print("Training: Epoch[{:0>3}/{:0>3}], Loss: {:.4f} Dice:{:.2%}".format(
                epoch, MAX_EPOCH, loss_trn / iter_num, dice_trn / iter_num))

        # Validating
        loss_val = 0.; dice_val = 0.; iter_num = 0
        pred_val = []; label_val = []
        net.eval()
        with torch.no_grad():
            for image, label in valid_loader:
                iter_num += 1
                
                image = image.to(device)
                label = label[:,0,:,:].reshape([1] + list(label[:,0,:,:].shape))
                label = label.to(device)
                output = net(image)
                pred = output[0]
                
                if epoch == MAX_EPOCH:
                    pred_val.append(pred); label_val.append(label)
                
                loss_iou = IOU_loss(pred, label)
                loss = loss_iou #+ msssim(pred, label) + BCE_loss(pred + 1e-6, label)
                loss_val += loss.item()
                dice_val += 2 * loss_iou / (loss_iou + 1)
        
            print("Valid: Epoch[{:0>3}/{:0>3}], Loss: {:.4f} Dice:{:.2%}\n".format(
                epoch, MAX_EPOCH, loss_val / iter_num, dice_val / iter_num))

    print('\nTraining finish, the time consumption of {} epochs is {}s\n'.format(MAX_EPOCH, round(time.time() - start)))
    
    return(pred_trn, label_trn, pred_val, label_val)

In [ ]:
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("\nRunning on:", device)

    if device == 'cuda':
        device_name = torch.cuda.get_device_name()
        print("The device name is:", device_name)
        cap = torch.cuda.get_device_capability(device=None)
        print("The capability of this device is:", cap, '\n')
    
    # hyper-parameters
    seed = 1
    MAX_EPOCH = 30
    LR = 0.001
    weight_decay = 1e-3
    data_dir = 'segmented-images'
    
    set_seed(seed)
    print('random seed:', seed)
    main()

In [ ]:
torch.save(net, 'unet3plus.pth')

In [ ]:
torch.load('unet3plus.pth')